# UBS Challenge Submission

Group Name: "" (empty string)

Group Participants: ...

The culmination of your submission should be a technical report that outlines your analytical journey, highlighting the methodologies employed, any obstacles encountered along the way, and the strategies adopted to
overcome these challenges.

## 1. Data Understanding

### i. Data Cleaning and Preprocessing

Provide a detailed account of the initial steps taken to prepare the data for analysis.

This should include a description of how data quality issues, such as missing values or outliers, were addressed.

1. Exploratory Data Analysis
   - Examination and understanding of the dataset's structure and content
   - Performing exploratory data analysis to understand:
     - data patterns, 
     - outliers, and
     - relationships between variables

2. Data Cleaning
   - Data preprocessing, include, but not limited to:
     - handling missing values, 
     - data conversions, and 
     - normalization

### ii. Assumptions

Clearly articulate any assumptions that were made during the data preparation phase.

### iii. Feature Engineering and Data Augmentation

Describe any techniques employed to enhance the dataset, whether through the creation of new features or augmentation of the existing data.

Goal: create relevant features for identifying deviations

## 2. Model Approach

### i. Model Selection

Explain on the choice(s) of statistical or machine learning model(s) utilized in your analysis. Provide a compelling justification for each model selected, emphasizing how they align with the objectives of the challenge.

How could we go about solving this challenge?

### ii. Recommendation for Model Enhancement

Conclude with a thoughtful reflection on potential avenues for further improving your model. Propose specific modifications or additional analyses that could refine your predictions and insights.

## 3. Actionable Insights

In the process of finding outliers within the given dataset, we came to an understanding of the trends that govern social media performance of the given firms.

We need insights into:

1. identify outlier firms that are performing exceptionally well on social media
  
    -> this could indicate strong brand engagement and customer loyalty
    -> might translate into future profitability
    -> investment opportunities

    find why they are well-performing?
    
    possible causes:
    - compare to industry?
    - long & strong brand history
    - strong brand ambassadors
    - good ethics of company (planting trees etc)
    - funny social media accounts whatever

2. identify outlier firms on lower end

    -> signals issues with brand perception, customer engagement, or emerging crises
    -> poses risk to company's long-term stability 
    -> requires proactive (risk) management.

    why are they not well-performing?
    - industry?
    - scandals?
    - stock development?

3. identify market trends, industry trends

    - this is actually more of the data exploration part, but we need to synthesize the findings into the insights report because other groups might not do it, so it gives us an edge
  
    -> can indicate broader market trends / shifts in consumer behaviour
    -> can help more informed broadcasting & market analysis
    -> more of a tool to use as input to more complex market models
    -> can probably work well together with recent LLMs that consume market news/sentiment because social media is a good representation of what people think

4. identify actual weird outlier firms -> can generate solid investment advice, probably

    -> idea with k-means clustering: have them clustered according to performance features
    -> then find "bad stock performance" companies in "good social media performance" companies and vice versa

5. general UBS use of our analysis: advise their customers why their competitors are outliers, which can help the clients refine their competitive strategies. Identifying key factors leading to higher engagement can inform better marketing strategies. & also: Product and Service Development: Insights derived from social media performance can influence decisions about product innovations or adjustments, focusing on areas that resonate with or displease the market.